## xcube Data Cube Generator Service

This notebook provides a walk-through demonstrating how to use the _data cube generator service_.

An introduction to the xcube data cube generators can be found in the [Getting Started](./1_getting_started.ipynb) Notebook.
The data cube generator service demonstrated here is a Python client for the [xcube Generator service API](https://stage.xcube-gen.brockmann-consult.de/api/v2/ui/).

Service processing units and credentials can be obtained by purchasing the API Service **xcube Generator** from the [EuroDataCube Marketplace](https://www.eurodatacube.com/marketplace).

The Python client is represented by the class `CubeGeneratorService`. Instances of this are created by passing it a 
`CubeGeneratorRequest` and a `ServiceConfig` instance. Here are imports for the named classes:

In [ ]:
from xcube.core.gen2 import CubeGeneratorRequest
from xcube.core.gen2.service import CubeGeneratorService
from xcube.core.gen2.service import ServiceConfig

We first need to provide `client_id` and `client_secret` to authorize for using the service. 
An `endpoint_url` is optional and defaults to the standard endpoint for the EuroDataCube cube generation service.

In [ ]:
service_config_json = {
    "client_id": "...",
    "client_secret": "..."
}

However, it is much saver to ingest service configuration from a JSON or YAML file (the file path used must be adjusted):

In [ ]:
import yaml
with open('../../../edc-service.yml') as fp:
    service_config_json = yaml.safe_load(fp)

Validate and convert `service_config_json` to a service configuration object:

In [ ]:
service_config = ServiceConfig.from_dict(service_config_json)

Now we define the _cube generator request_. 
The following example is supposed to read an `S2L2A` dataset from the Sentinel Hub data store (`input_config`), 
using numerous target cube parameters(`cube_config`), and
writes the target cube to a well-known scratch bucket in AWS S3 (`output_config`).

This is how a _cube generator request_ looks as a (JSON) dictionary:

In [ ]:
request_json = {
  "input_config": {
    "store_id": "@sentinelhub_eu",
    "data_id": "S2L2A"
  },
  "cube_config": {
    "variable_names": ["B04", "B05"],
    "bbox": [7, 53, 9, 55],
    "spatial_res": 0.001,
    "tile_size": [1024, 1024],
    "crs": "WGS84",
    "time_range": ["2019-08-05", "2019-08-10"],
    "time_period": "1D"
  },
  "output_config": {
    "store_id": "@eurodatacube_scratch",
    "data_id": "S2L2A_B04_B05_7_53_9_55.zarr",
    "replace": True
  }
}

Validate and convert `request_json` to a request object:

In [ ]:
request = CubeGeneratorRequest.from_dict(request_json)
request

Instantiate the generator object:

In [ ]:
gen = CubeGeneratorService(request, service_config, verbosity=1)

Get some information about the cube that would be generated by `gen` (this may take several seconds to complete):

In [ ]:
cube_info = gen.get_cube_info()
cube_info

Now perform the actual cube generation (this may take up to a minute):

In [ ]:
cube_id = gen.generate_cube()
cube_id

Let's open the generated cube. Cubes generated into the store "@eurodatacube_scratch" 
can be accessed via URL template "https://s3.eu-central-1.amazonaws.com/eurodatacube-scratch/{cube_id}":

In [ ]:
from xcube.core.dsio import open_cube
cube = open_cube(f'https://s3.eu-central-1.amazonaws.com/eurodatacube-scratch/{cube_id}')
cube

_This is a work in progress. More material will follow in an upcoming xcube release._